### Steps to preprocess FIADB data for use in `synthetic-knn`

This is a walkthrough for getting data from the Forest Inventory and Analysis Database (FIADB) into a format that can be used in the `synthetic-knn` package. Data for FIADB is hosted at the [FIA DataMart](https://apps.fs.usda.gov/fia/datamart/datamart.html) and for this example, we will be downloading data for the state of Oregon from 2011-2020.  This constitutes a full cycle of plots (10 years).

The first step is to download the data from the FIA DataMart.  Select `SQLite` format and choose "Oregon" in the "Select State/s" dropdown.  In the "Data Available for Download" section, click on the "SQLite_FIADB_OR.zip" link to download the data (you can also download the Oregon database directly at [this link](https://apps.fs.usda.gov/fia/datamart/Databases/SQLite_FIADB_OR.zip)).

This will download a zip file that contains the SQLite database.  Open this zip file in an analysis directory such that the SQLite database is called "SQLite_FIADB_OR.db", then follow the below steps to create the required CSV files for the `synthetic-knn` package.

#### Brief overview of the FIADB database structure

The FIADB database is a relational database with many tables.  The only two tables that we use in this walkthrough are the `PLOT` and `TREE` tables.  The `PLOT` table contains field collection details, including an approximate location of the plot, although not the exact location of the plot in order to preserve plot confidentiality.  A primary key (`CN`) is used to uniquely identify each plot in the database.

The `TREE` table contains tree data (both live and standing dead) for each plot, including attributes such as tree species, diameter at breast height (DBH), height, and other attributes.  This table also has a primary key (`CN`) that uniquely identifies each tree in the database.  The `PLOT_CN` field is a foreign key that links each tree to the `CN` field in the `PLOT` table.

#### Step 1: Install the required packages

For this workflow, you should only need to install the `pandas`, `pyproj`, and `rasterio` packages into your current environment (note that `numpy` is a dependency of both `pandas` and `rasterio`, so it doesn't need to be installed separately).  The `sqlite3` package is included with Python by default.  You can execute the following command to install `pandas`, `pyproj`, and `rasterio` into your current Python environment:

In [ ]:
%pip install pandas
%pip install pyproj
%pip install rasterio

#### Step 2: Extract the tree data from the `TREE` table in the SQLite database

We will be subsetting the tree data using the following filters:

- `TREE.STATUSCD` is 1 (live trees)
- `TREE.INVYR` is between 2011 and 2020
- `COND.COND_STATUS_CD` is 1 (forested)
- `COND.CONDPROP_UNADJ` is 1.0 (single-condition plots)

Assuming that your `SQLite_FIADB_OR.db` file is in the same directory as this notebook, you can execute the following cell to extract the tree data from the `TREE` table in the SQLite database.  Note that if you receive the error `OperationalError: unable to open database file`, make sure that you've extracted the SQLite database from the zip file and that it is in the same directory as this notebook.

In [ ]:
# Import modules and set up the connection to the database
import sqlite3

import pandas as pd
from pyproj import Transformer

# Connect to the SQLite database
connection = sqlite3.connect("file:./SQLite_FIADB_OR.db?mode=ro", uri=True)

In [ ]:
sql = """
    -- Get all live tree records for full-plot forested conditions
    -- between 2011 and 2020
    select
      t.CN AS TREE_CN,
      t.PLT_CN,
      cast(t.SPCD as INT) as SPCD,
      t.DIA,
      t.TPA_UNADJ
    from
      tree t
    join
      plot p on t.PLT_CN = p.CN
    join
      cond c on t.PLT_CN = c.PLT_CN and t.CONDID = c.CONDID 
    where
      t.STATUSCD = 1
      and c.CONDPROP_UNADJ = 1.0
      and c.COND_STATUS_CD = 1
      and p.INVYR >= 2011
      and p.INVYR <= 2020
    order by t.CN
"""
tree_df = pd.read_sql_query(sql, connection)

This should produce a pandas `DataFrame` with n=166,147 records and 5 columns.

In [ ]:
print(tree_df.head())
print(f"Tree table shape: {tree_df.shape}")

#### Step 3: Extract the associated plot data and coordinates from the `PLOT` table in the SQLite database

FIADB stores fuzzed and swapped coordinates in the `PLOT` table.  As such, this data should not be used in any actual analysis as the coordinates are not accurate. Nonetheless, this workflow will illustrate the needed format of the files.  Note that we are extracting the measurement year (`MEASYEAR`) rather than the inventory year (`INVYR`) as this will be a closer temporal match against spatial data that we use in our modeling workflow.  Run the following cell to create a plot CSV file. 

In [ ]:
# Plot coordinates
sql = """
    -- Get all plot records for full-plot forested conditions
    -- between 2011 and 2020
    select
      p.CN as PLT_CN,
      p.MEASYEAR as ASSESSMENT_YEAR,
      p.LON,
      p.LAT
    from
      plot p
    join
      cond c on p.CN = c.PLT_CN
    where
      c.CONDPROP_UNADJ = 1.0
      and c.COND_STATUS_CD = 1
      and p.INVYR >= 2011
      and p.INVYR <= 2020
    order by p.CN
"""
plot_df = pd.read_sql_query(sql, connection)

This produces a pandas `DataFrame` with n=5,671 records and 4 columns.

In [ ]:
print(plot_df.head())
print(f"Plot table shape: {plot_df.shape}")

At this point, we are done with the database and can close the connection.

In [ ]:
connection.close()

#### Step 4: Transform the data to deal with unit conversions, species codes, and projected coordinates

For use in `synthetic-knn`, we make the following assumptions about the input tree data:

- Units are metric (e.g. diameter in cm, density in trees per hectare)
- Species are represented by USDA PLANTS database codes
- Unique tree and plot IDs are stored as integers

In order to crosswalk FIA species codes to USDA PLANTS database codes, we create a crosswalk from FIA numeric species codes to [USDA PLANTS](https://plants.usda.gov/) symbols.  FIA provides a reference table called `REF_SPECIES.csv` as part of its [FIADB Reference Table CSV Archive](https://apps.fs.usda.gov/fia/datamart/CSV/FIADB_REFERENCE.zip) which we've put into the `data` subdirectory.  The FIA numeric species code is in the `SPCD` column and the PLANTS symbol is in the `SPECIES_SYMBOL` column.

We also need to transform the geographic coordinates from FIADB to the reference projection of our spatial data (USGS National Albers).  We will modify the `tree_df` and `plot_df` dataframes in place to make these changes.

In [ ]:
xwalk_df = pd.read_csv("./data/REF_SPECIES.csv")
FIA_TO_PLANTS_XWALK = pd.Series(xwalk_df.SPECIES_SYMBOL.values, index=xwalk_df.SPCD).to_dict()

# Define a transformer from NAD-83 to USGS National Albers
transformer = Transformer.from_crs("EPSG:4269", "EPSG:5070", always_xy=True)

def transform_coordinates(row: pd.Series):
    return pd.Series(transformer.transform(row.LON, row.LAT))

def set_cn_lookup(df: pd.DataFrame, field: str) -> dict:
    return dict(zip(df[field], range(1, len(df) + 1)))

def transform_plot_data(
    df: pd.DataFrame,
    plt_cn_dict: dict[str, int],
    inplace: bool = True
) -> pd.DataFrame:
    if not inplace:
        df = df.copy()

    df["PLOT_ID"] = df.PLT_CN.map(plt_cn_dict)
    df[["X", "Y"]] = df.apply(transform_coordinates, axis=1)

    if not inplace:
        return df

def transform_tree_data(
    df: pd.DataFrame,
    plt_cn_dict: dict[str, int],
    inplace: bool = True
) -> pd.DataFrame:
    if not inplace:
        df = df.copy()

    df["TREE_ID"] = df.TREE_CN.map(set_cn_lookup(df, "TREE_CN"))
    df["PLOT_ID"] = df.PLT_CN.map(plt_cn_dict)
    df["SPECIES_SYMBOL"] = df.SPCD.map(FIA_TO_PLANTS_XWALK)
    df["DBH_CM"] = df.DIA * 2.54
    df["TPH"] = df.TPA_UNADJ / 0.404686

    if not inplace:
        return df

# Create sequential plot IDs
plt_cn_dict = set_cn_lookup(plot_df, "PLT_CN")

# Transform the dataframes and subset to the relevant fields
transform_plot_data(plot_df, plt_cn_dict, inplace=True)
transform_tree_data(tree_df, plt_cn_dict, inplace=True)

# Subset the dataframes to the relevant fields
plot_df = plot_df[["PLOT_ID", "PLT_CN", "ASSESSMENT_YEAR", "X", "Y"]]
tree_df = tree_df[["TREE_ID", "TREE_CN", "PLOT_ID", "SPECIES_SYMBOL", "DBH_CM", "TPH"]]

# Show the first few rows of each dataframe
print(plot_df.head())
print(tree_df.head())

#### Step 5: Create a species by DBH-class matrix of basal area for CCA modeling

Canonical Correspondence Analysis (CCA) is the ordination method used with the Gradient Nearest Neighbor (GNN) method and needs both a species matrix and an environmental matrix (next step) to run.  Typically, we have used species by 25-cm diameter classes to create the species matrix.

It's also important to realize that there may be some plots that don't have any tree records associated with them.  This typically happens when a plot has been recently disturbed either through harvest or fire.  Because the *land use* is still in a forested condition, we want to include these plots in our analysis.  However, in order for CCA to run correctly, each plot (row) must have a non-zero sum of basal area.  On these plots without trees, we create a separate species called `NOTALY` and assign it a basal area of 0.001 m^2 ha^-1.  

We can create this matrix from `tree_df` by running the following cell:

In [ ]:
def create_species_matrix(tree_df: pd.DataFrame, plot_df: pd.DataFrame) -> pd.DataFrame:
    df = tree_df.copy()

    # Add tree basal area and basal area per hectare columns
    df["BA_M2"] = (df.DBH_CM ** 2) * 0.00007854
    df["BAPH"] = df.BA_M2 * df.TPH

    # Create a new column called DBH_GROUP which groups the DBH_CM column
    # these increments [<25, 25-50, 50-75, 75-100, >100].  Call these classes
    # 1 - 5.
    dbh_bins = [0, 25, 50, 75, 100, 1000]
    df["DBH_GROUP"] = pd.cut(df.DBH_CM, bins=dbh_bins, labels=[1, 2, 3, 4, 5])

    # Combine the SPECIES_SYMBOL and DBH_GROUP fields into a new column
    # called SPECIES_SIZE_CLASS separated by an underscore
    df["SPECIES_SIZE_CLASS"] = df["SPECIES_SYMBOL"] + "_" + df["DBH_GROUP"].astype(str)

    # Group by this new column and sum the BAPH column
    grouped_df = df.groupby(["PLOT_ID", "SPECIES_SIZE_CLASS"]).BAPH.sum()

    # Pivot the data into a matrix
    grouped_df = grouped_df.reset_index()
    grouped_df = grouped_df.pivot(
        index="PLOT_ID", columns="SPECIES_SIZE_CLASS", values="BAPH"
    )

    # Merge plot_df with the grouped_df to account for plots with no tree
    # records and fill the missing NA values with 0
    grouped_df = plot_df[["PLOT_ID"]].merge(grouped_df, on="PLOT_ID", how="left")
    grouped_df = grouped_df.fillna(0)

    # Get the sum of all but the PLOT_ID column to find plots with no tree records
    row_sums = grouped_df.iloc[:,1:].sum(axis=1)

    # If row_sums is 0, set a NOTALY field to 0.001, otherwise set to 0
    grouped_df["NOTALY"] = row_sums.apply(lambda x: 0.001 if x == 0 else 0)

    return grouped_df

spp_size_df = create_species_matrix(tree_df, plot_df)
print(spp_size_df)

#### Step 6: Create an environmental matrix for CCA modeling by extracting values at plot locations

Because each user's area of interest will be different and storing rasters in this repository is not feasible, we describe the actions needed to create the environmental matrix as well as provide example code for users to create their own.  For the test data as part of this repository, we performed the following steps:

- Created a library of 30m resolution rasters with the same extent and registration.  These rasters included climate, topography, location, and Landsat-derived spectral information.  Codes in the output file are:
  - `ANNPRE`: Annual precipitation
  - `ANNTMP`: Mean annual temperature
  - `AUGMAXT`: Mean August maximum temperature
  - `DECMINT`: Mean December minimum temperature
  - `SMRTP`: Growing season moisture stress
  - `ASPTR`: Beers' transformed aspect
  - `DEM`: Elevation
  - `PRR`: Potential relative radiation
  - `SLPPCT`: Slope percent
  - `TPI450`: Topographic position index
  - `LAT`: Latitude
  - `LON`: Longitude
  - `NBR`: Normalized burn ratio
  - `TC1`: First component of Tasseled Cap transformation
  - `TC2`: Second component of Tasseled Cap transformation
  - `TC3`: Third component of Tasseled Cap transformation

- For each plot in `plot_df`, we first snapped the plot coordinate to the center of its enclosing pixel, then created a 90m x 90m window centered on the snapped plot location.  For each raster, we extracted the mean value of the nine pixels in this window.  For covariates that vary by year (`TC1`, `TC2`, `TC3`, `NBR`), we matched the image year to the plot measurement year.  The below code demonstrates this workflow, although the raster paths are not valid and users will need to install both the `rasterio` and `numpy` packages to run this code (note that this cell can take a while to run and is not intended to be run as part of this notebook).)

The output of this code is a pandas `DataFrame` with n=5,671 records and 17 columns.

In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window

# Define the paths to the raster files
# Users will need to update these paths to match their local file structure
RASTERS = {
    "ANNPRE": "./annpre.tif",
    "ANNTMP": "./anntmp.tif",
    "AUGMAXT": "./augmaxt.tif",
    "DECMINT": "./decmint.tif",
    "SMRTP": "./smrtp.tif",
    "ASPTR": "./asptr.tif",
    "DEM": "./dem.tif",
    "PRR": "./prr.tif",
    "SLPPCT": "./slppct.tif",
    "TPI450": "./tpi450.tif",
    "LAT": "./lat.tif",
    "LON": "./lon.tif",
    f"NBR": "./nbr_{year}.tif",
    f"TC1": "./tc1_{year}.tif",
    f"TC2": "./tc2_{year}.tif",
    f"TC3": "./tc3_{year}.tif",
}

def get_rasters(columns: list[str], year: int) -> list[rasterio.DatasetReader]:
    """Open the raster files for the given columns and year."""
    return [rasterio.open(RASTERS[col].format(year=year)) for col in columns]

def get_values_at_footprint(rasters: list[rasterio.DatasetReader], coord: tuple[float]) -> list[float]:
    """Get the values at the 3x3 pixel footprint of the given coordinates for each raster."""
    windows = [Window(*r.index(coord[0], coord[1])[::-1], 3, 3) for r in rasters]
    return np.array([r.read(1, window=window).flatten() for r, window in zip(rasters, windows)]).T

def extract_footprints(row: pd.Series, rasters: list[rasterio.DatasetReader]):
    """Wrapper around get_values_at_footprint to extract the values for a row."""
    ul_window_coord = (row.X - 30.0, row.Y + 30.0)
    values = get_values_at_footprint(rasters, ul_window_coord)
    return pd.Series(data=[row.PLOT_ID] + list(values.mean(axis=0)), index=["PLOT_ID"] + raster_keys)

raster_keys: list[str] = list(RASTERS.keys())
years = sorted(plot_df["ASSESSMENT_YEAR"].unique())
dfs = []

for year in years:
    print(year)
    rasters = get_rasters(raster_keys, year)
    year_df = plot_df[plot_df.ASSESSMENT_YEAR == year]
    dfs.append(year_df.apply(extract_footprints, axis=1, rasters=rasters))
env_df = pd.concat(dfs).sort_values("PLOT_ID")
env_df["PLOT_ID"] = env_df.PLOT_ID.astype(int)

print(env_df)

#### Step 7: Create CSV files of tree, plot, species, and environmental data

We write the `tree_df`, `plot_df`, `spp_size_df`, and `env_df` dataframes to CSV files.  These files will be used in the `synthetic-knn` package to create synthetic tree data.  The plot file is not necessarily needed for `synthetic-knn`, but we will keep it to retain the lookup between `PLOT_ID` and `PLT_CN`.  Run the following code to write the CSV files:

In [ ]:
tree_df.to_csv("./fiadb_oregon_live_tree_2011_2020.csv", index=False, float_format="%.4f")
plot_df.to_csv("./fiadb_oregon_plot_2011_2020.csv", index=False, float_format="%.4f")
spp_size_df.to_csv("./fiadb_oregon_species_size_2011_2020.csv", index=False, float_format="%.4f")
env_df.to_csv("./fiadb_oregon_environment_2011_2020.csv", index=False, float_format="%.4f")